# 학습된 NarrativeKoGPT2을 이용한 Text Generation

## 1.Google Drive 연동
- 모델 파일과 학습 데이터가 저장 되어있는 구글 드라이브의 디렉토리와 Colab을 연동. 

### 1.1 Google Drive 연동
아래 코드를 실행후 나오는 URL을 클릭하여 나오는 인증 코드 입력

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Colab 디렉토리 아래 NarrativeKoGPT2 경로 확인**

In [3]:
!ls drive/'My Drive'/'Colab Notebooks'/

BERT_X	KorQuAD-beginner  NarrativeKoGPT2


**필요 패키지들 설치**

In [4]:
!pip install -r drive/'My Drive'/'Colab Notebooks'/NarrativeKoGPT2/requirements.txt

     |████████████████████████████████| 256kB 2.7MB/s 
     |████████████████████████████████| 68.7MB 44kB/s 
     |████████████████████████████████| 1.0MB 42.6MB/s 
     |████████████████████████████████| 501kB 52.7MB/s 
     |████████████████████████████████| 870kB 43.9MB/s 
     |████████████████████████████████| 3.7MB 37.1MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470973 sha256=ce0a0e8bff9f39914afe70e85415bf087a6dfdc32ee2642f8e8623e0d2847c2d
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=b304eeacb4ed3c89b77e1e08a51c7e5176adbcfcd8c0bb1a7523d252ae36192f
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built gluonnlp sacremoses
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
     

**시스템 경로 추가**

In [0]:
import os
import sys
sys.path.append('drive/My Drive/Colab Notebooks/')

## 2.KoGPT2 Text Generation

### 2.1.Import Package

In [6]:
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer 
from NarrativeKoGPT2.kogpt2.utils import get_tokenizer
from NarrativeKoGPT2.kogpt2.utils import download, tokenizer
from NarrativeKoGPT2.model.torch_gpt2 import GPT2Config, GPT2LMHeadModel
from NarrativeKoGPT2.util.data import NovelDataset
import gluonnlp

### 2.2. koGPT-2 Config

In [0]:
ctx= 'cpu'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
epoch =200  # 학습 epoch
save_path = 'drive/My Drive/Colab Notebooks/NarrativeKoGPT2/checkpoint/'
load_path = 'drive/My Drive/Colab Notebooks/NarrativeKoGPT2/checkpoint/narrativeKoGPT2_checkpoint.tar'
#use_cuda = True # Colab내 GPU 사용을 위한 값

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000
}

### 2.3 Model and Vocab Download

In [8]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


### 2.4.KoGPT-2 Model Vocab

**추론 및 학습 재개를 위한 모델 불러오기**
**저장하기**
```python
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            ...
            }, PATH)

```
  
**불러오기**
``` python
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()
```

In [0]:
# Device 설정
device = torch.device(ctx)
# 저장한 Checkpoint 불러오기
checkpoint = torch.load(load_path, map_location=device)

# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
kogpt2model.load_state_dict(checkpoint['model_state_dict'])

kogpt2model.eval()
vocab_b_obj = gluonnlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')


### 2.5. Tokenizer

In [11]:
tok_path = get_tokenizer()
model, vocab = kogpt2model, vocab_b_obj
tok = SentencepieceTokenizer(tok_path)

using cached model


### 2.6. NarrativeKoGPT-2 Text Generation

In [24]:
sent =''
while 1:

  tmp_sent = input('다음...: ')
  sent = sent+tmp_sent

  toked = tok(sent)
  count = 0
  generated_text =''
  input_size = 50

  if len(toked) >1022:
    break

  while 1:
    input_ids = torch.tensor([vocab[vocab.bos_token],]  + vocab[toked]).unsqueeze(0)
    predicts = model(input_ids)
    pred = predicts[0]
    # print('predicts:', torch.argmax(pred, axis=-1).squeeze())
    gen = vocab.to_tokens(torch.argmax(pred, axis=-1).squeeze().tolist())[-1]
    if gen == '</s>':
      print('to_tokens:',vocab.to_tokens(torch.argmax(pred, axis=-1).squeeze().tolist()))
    if gen == '.' or count>input_size:
      sent += gen.replace('▁', ' ')
      generated_text += gen.replace('▁', ' ')
      sent += '\n'
      generated_text += '\n'
      toked = tok(sent)
      count =0
      break
      # print('to_tokens:',vocab.to_tokens(torch.argmax(pred, axis=-1).squeeze().tolist()))
    # if count >= input_size:
    #   break
    sent += gen.replace('▁', ' ')
    generated_text += gen.replace('▁', ' ')
    # print(generated_text)

    toked = tok(sent)
    count += 1
  print(generated_text)
  generated_text=''
print(sent)

다음...: 아니라고 말하는게 
 더 나아.

다음...: 더 낫다니
."민욱은 윤애를 돌아봤다.

다음...: 오잉??
 윤애가 놀란건가? 윤애가 민욱을 흘끗 쳐다봤다."왜?""너, 주민욱이랑 사귀는거아니었어?""......""왜? 사귀는거아니었다고?""응,       

다음...: 내가 주민욱이랑 왜 사귀는데??
 사귀는사람이 날 좋아하는데?""날 좋아한다는 이유로 사귀는게아니었다고?""응, 난 널 좋아했었어.""......""그래서 난 널 사랑해.""......""그래서 난 널 사랑해."민욱의 입꼬리가 슬쩍 올라갔다."그런데 왜

다음...: 그딴거 다 집워치우고
 윤애랑 도망친거야? 응? 사귀는사람이 날 좋아한다는 이유인데, 왜그랬어? 응? 그런데 왜그랬어? 응? 그런데 왜그랬냐고? 왜그랬냐고? 왜그랬냐고?"민욱이

다음...: 제발 같은 말 반복하게 하지마 나는 킷캣이 더 좋아
."민욱의 팔에 얼굴을 묻었다.

다음...: 킷캣은 아주 멋진 친구야
.

다음...: 하지만 말이야
, 넌 그렇지않아.

다음...: 나는 그렇지 않아도 말이야 그가 나를
 정말로 좋아하는지 아닌지는 알수없거든.

다음...: 
 하지만                                                   

다음...: 하지만 뭔데!
                                                    

다음...: 왜 예측하지 못하는거야
? 왜 하필이면                                               

다음...: 그러게..
...?                                                  

다음...: 지켜보는 우혁의 가슴을 갈기갈기 찢어놓았다. "으...............흐흑.................흑......................." 힘겹게 울었다. 목 놓아 울지도 못하고
, 입술을 깨물지도못했다.

다음...: 구슬 같은 눈물만 뚝뚝

KeyboardInterrupt: ignored